In [963]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
import sqlite3
import json
import numpy as np
import os
import pickle
import shutil
import sqlite3
from collections import defaultdict
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret
import warnings
from pandas.core.common import SettingWithCopyWarning
from logzero import logger
from pandas.tseries.offsets import MonthEnd
import random
from requests import get
import calendar
import regex as re
import pandas as pd
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [965]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [966]:
query_c_grab = """
SELECT zipcode

FROM g_df2
"""

conn_proj = sqlite3.connect("../data/Berkeley_2.db")
cursor_proj = conn_proj.cursor()

df = pd.read_sql_query(query_c_grab, conn_proj)
data_zips = df.zipcode.unique()

conn_proj.close



<function Connection.close>

In [967]:
query_types = """
SELECT *

FROM program_type
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

program_types = pd.read_sql_query(query_types, conn_proj)
    
conn_proj.close

<function Connection.close>

In [968]:
programs = list(program_types.name)
programs

['Solar Access Law/Guideline',
 'Green Building Incentive',
 'Mandatory Utility Green Power Option',
 'Performance-Based Incentive',
 'Interconnection',
 'Corporate Tax Credit',
 'Corporate Tax Deduction',
 'Solar/Wind Contractor Licensing',
 'Corporate Depreciation',
 'Equipment Certification',
 'Generation Disclosure',
 'Personal Tax Credit',
 'Personal Tax Deduction',
 'Personal Tax Exemption',
 'Line Extension Analysis',
 'Net Metering',
 'Renewables Portfolio Standard',
 'Industry Recruitment/Support',
 'Solar/Wind Access Policy',
 'Public Benefits Fund',
 'Corporate Tax Exemption',
 'Property Tax Assessment',
 'Appliance/Equipment Efficiency Standards',
 'Utility Rate Discount',
 'Energy Standards for Public Buildings',
 'Building Energy Code',
 'Solar/Wind Permitting Standards',
 'Other Incentive',
 'Other Policy',
 'PACE Financing',
 'Property Tax Incentive',
 'Green Power Purchasing',
 'Leasing Program',
 'Sales Tax Incentive',
 'Energy Efficiency Resource Standard',
 'Solar R

In [969]:
# select type - map to id
p_type = 'Personal Tax Deduction'

def pol_map(p_type):
    
    query_type_ind = """
    SELECT id

    FROM program_type
    
    WHERE program_type.name = ?
    """

    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()

    program_code = pd.read_sql_query(query_type_ind, conn_proj,params = [p_type])
    #program_code = cursor_proj.execute(query_type_ind, [programs[0]])
    
    conn_proj.close
    return int(program_code.id)
    

val = pol_map(p_type)

val

32

In [970]:
### filter input month 

In [971]:
def zip_find(val):
    query_zip_find = """
    SELECT zipcode

    FROM programs_map
    
    WHERE programs_map.program_type_id = ?
    
    AND programs_map.nb_months >= 37
    """

    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()

    zc = pd.read_sql_query(query_zip_find, conn_proj,params = [val])
    #program_code = cursor_proj.execute(query_type_ind, [programs[0]])
    
    conn_proj.close
    return list(zc.zipcode)

all_zips = zip_find(val)

In [972]:
a_zips = list(set(data_zips).intersection(set(all_zips)))


In [973]:
zip_in = random.sample(a_zips,1)[0]
print(zip_in)

def zip_map(val, zip_in):
    
    query_zips_ind = """
    SELECT *

    FROM programs_map
    
    WHERE programs_map.program_type_id = ? 
    AND programs_map.zipcode = ?
    
    
    """

    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()

    df = pd.read_sql_query(query_zips_ind, conn_proj,params = [val, zip_in])
    #program_code = cursor_proj.execute(query_type_ind, [programs[0]])
    
    conn_proj.close
    return list(df.start_date)

dates_in = zip_map(val, zip_in)

94024


In [974]:
dates_in[0]


'2001-10-01 00:00:00'

In [975]:
date_in = dates_in[-1]

def zip_elim(val,date_in):
       
    query_zips_elim = """
    SELECT *

    FROM programs_map
    
    WHERE programs_map.program_type_id = ? 
    AND programs_map.start_date = ?
    
    """

    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()

    df = pd.read_sql_query(query_zips_elim, conn_proj,params = [val, date_in])
    #program_code = cursor_proj.execute(query_type_ind, [programs[0]])
    
    conn_proj.close
    #df2 = df[df.zipcode != zip_in]
    elim_zips = list(df.zipcode)
    #e_zips = elim_zips.remove(zip_in)
    month = df.nb_months.unique()[0]
    
    return elim_zips, month
    
elim_zips, month = zip_elim(val,date_in)

In [976]:
fil_zips = elim_zips.copy()
fil_zips.remove(zip_in)

In [977]:
def date_grab(month,fil_zips, zip_in):
    
    
    query_g_diff = """
    SELECT zipcode, price, NREL_PSM_2019, cost_electricity, rolling_slope

    FROM g_df2
    
    WHERE g_df2.month = ? AND g_df2.zipcode NOT IN ({seq})

    """.format(seq=','.join(['?']*len(fil_zips)))
    
    conn_proj = sqlite3.connect("../data/Berkeley_2.db")
    cursor_proj = conn_proj.cursor()

    comp_df = pd.read_sql_query(query_g_diff, conn_proj, params = [float(month - 1)] + fil_zips)
    comp_df['cost_electricity'] = comp_df['cost_electricity'].astype(float)
    conn_proj.close
    return comp_df
comp_df = date_grab(month,elim_zips, zip_in)

In [978]:

s_df = comp_df.drop(columns = 'zipcode')
s_df.head()

,price,NREL_PSM_2019,cost_electricity,rolling_slope
0,9993.61,7.118120,8.81,0.0
1,9993.61,7.487866,8.81,0.0
2,9993.61,7.265153,8.81,0.0
3,9993.61,7.242748,8.81,0.0
4,9993.61,7.297422,8.81,0.0


In [979]:
#index_in = comp_df[comp_df.zipcode == zip_in].index[0]
comp_df[comp_df.zipcode == zip_in]

,zipcode,price,NREL_PSM_2019,cost_electricity,rolling_slope


In [980]:


from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(s_df.values)

scaled_features_df = pd.DataFrame(scaled_features, index=s_df.index, columns=s_df.columns)

df1 = scaled_features_df.iloc[index_in].to_frame().T
df2 = scaled_features_df.iloc[lambda x: x.index != index_in]

In [981]:
import scipy
ary = scipy.spatial.distance.cdist(df2, df1, metric='euclidean')
index_out = df2[ary==ary.min()].index[0]
comp_zip = list(comp_df.iloc[index_out].to_frame().T.zipcode)[0]
comp_zip

'75205'

In [982]:
def comp_grab(month,zip_in,comp_zip):
    

    query_c_grab = """
    SELECT zipcode, month, price, NREL_PSM_2019, cost_electricity, i_count

    FROM g_df2
    
    WHERE g_df2.zipcode IN (?,?)

    """
    
    conn_proj = sqlite3.connect("../data/Berkeley_2.db")
    cursor_proj = conn_proj.cursor()

    diffndiff_df = pd.read_sql_query(query_c_grab, conn_proj, params = [zip_in,comp_zip])
    diffndiff_df['cost_electricity'] = diffndiff_df['cost_electricity'].astype(float)
    
    output = diffndiff_df[(diffndiff_df.month <= float(month + 12)) & (diffndiff_df.month >= float(month - 12))]
    conn_proj.close
    return output
dd_df = comp_grab(month,zip_in,comp_zip)

dd_df['sort_v'] = dd_df.zipcode.map({zip_in:0,comp_zip:1})
dd_df = dd_df.sort_values(by = ['sort_v', 'month'])

In [983]:
dd_df

,zipcode,month,price,NREL_PSM_2019,cost_electricity,i_count,sort_v
1,94024,36.0,10304.460000,6.570772,10.89,0.0,0
3,94024,37.0,10265.300000,6.570772,11.14,0.0,0
5,94024,38.0,10226.230000,6.570772,11.29,0.0,0
7,94024,39.0,10187.250000,6.570772,11.14,0.0,0
9,94024,40.0,10148.350000,6.570772,11.97,0.0,0
11,94024,41.0,10109.540000,6.570772,12.87,0.0,0
13,94024,42.0,10070.810000,6.570772,13.05,0.0,0
15,94024,43.0,10032.170000,6.570772,12.80,0.0,0
17,94024,44.0,9993.610000,6.570772,12.87,0.0,0
19,94024,45.0,9835.089474,6.570772,12.49,1.0,0


In [984]:
import plotly.express as px 
fig = px.line(dd_df, x='month', y='i_count', color='zipcode')
fig.add_vline(x = month, line_dash = 'dot', annotation_text = 'Program Start: {}'.format(zip_in),annotation_position = 'top left')

In [985]:
zip_in in data_zips

True